# Examining the Choice of Kernel

This notebook examines how the choice of kernel affects a Gaussian Process (GP) forecast of LFPR. By specifying the kernel, one can implicitly determine the structure of the function that the GP forecasts. Previous efforts to forecast LFPR have imposed particular structures *a priori* without examining how this structure affects forecast performance. I consider a variety of different potential structures based on those employed in prior efforts to forecast LFPR and plot the resulting forecasts here. 

## Gaussian Processes for Forecasting

We start by treating the LFPR $y$ within an age-gender-year cell as a noisy observation from an unknown function of covariates $y = f(X) + \epsilon$ where the noise is normally distributed, $\epsilon ~ N(0, \sigma ^ 2)$. The unknown function $f$ is assumed to follow a Gaussian Process with a covariance kernel $K(X,X^{'})$. 

The choice of covariance kernel implicitly imposes structure on $f$. As an example, some kernels are stationary, which implies that these kernels will be mean-reverting, while other kernels are non-stationary. Some kernels, like the squared-exponential kernel, imply infinitely-differentiable functions, while others imply finitely-differentiable or non-differentiable functions. Further, since the sums and products of covariance kernels are covariance kernels themselves, complicated structure can be imposed by composing an overall kernel for the function $f$ from many individual kernels. 

As an example of how to impose structure through kernels, take the analysis of recent trends in LFPR contained in Krueger (2017). He estimates linear trends from 2000-2007 for sixteen demographic groups and projects these trends out through the Great Recession to examine how much of the post-2007 decline could be explained by continuation of these pre-existing trends. This method imposes the structure that 1) trends in LFPR over time are linear, and 2) similar ages are likely to have similar LFPR (and hence similar trends). This structure can be captured by the product of a linear kernel over the time dimension with a squared-exponential kernel over the age dimension. Functions drawn from a prior with this composite covariance kernel inherit this structure. 

Expressing the structure of $f$ through the kernels is beneficial for comparative, expositional, and computational reasons. In addition to Krueger (2017), many statistical models of LFPR can be expressed as a composition of kernels, including the commonly used age, gender, and cohort fixed effects approach. This allows all models to be compared within a unified framework and makes it easier to understand the differences between models. This also eases computation, as many different models can be estimated on the same dataset just by changing the covariance kernel. 

Our method is not the first to approach this type of forecasting problem. Forecasting LFPR is conceptually similar to forecasting the yield curve, as both are high-dimensional but contain a clear structure evident in the data. A long literature has developed methods to forecast the yield curve in a stationary environment. Some methods impose particular functional forms justified by economic reasoning to reduce the dimension of the forecast, as in the Dynamic Nelson-Siegel methods developed in Diebold & Li (2006) and Diebold et al (2006). Other approaches allow for general functional forms, as in the Functional Autoregression method employed by Kowal et al (2017). We follow in the spirit of the latter approaches by allowing for a wide variety of functional forms. However, we depart from this literature by examining a nonstationary process, LFPR, for which previous methods cannot be applied. 

## Comparison of Kernels

I examine the performance of seven different possible kernel structures. Each kernel structure may contain the sum or product of other kernels. I employ only two types of base kernels, Linear and Squared-Exponential (SE). Linear kernels are non-stationary while SE kernels are stationary, and their product is non-stationary. Refer to [this page](https://en.wikipedia.org/wiki/Gaussian_process#Usual_covariance_functions) for the exact formula of each kernel.

For the benefit of exposition, instead of reporting the kernel structure below, I report the functional form implied by each kernel structure. In the definitions below, $\alpha(\cdot)$ and $\beta(\cdot)$ are functions defined over some combination of age $a$, gender $g$, year $t$, and cohort $c$ using a product of separate SE kernels over each dimension. Linear terms are created by taking the product of a SE kernel with a linear kernel, i.e. $SE(age,gender)*Linear(time) \Rightarrow \beta(a,g)t$. I also exploit the fact that addition of two covariance kernels implies a functional form equal to adding together the two functions implied by each kernel separately, which allows me to write the functions below as sums of distinct components. 

1. $\alpha(a,g)+\beta(a,g)t$ - This estimates linear trends that vary across age/gender, similar to Krueger (2017). However, instead of imposing that the coefficients $\alpha(a,g)$ and $\beta(a,g)$ are identical within 10-year age groups and independent across groups, this functional form allows the coefficients to smoothly across ages according to an SE kernel. 

2. $\alpha(a,g,t)+\beta(a,g,t)t$ - This estimates linear trends across age/gender, but allows these trends to change slowly over time. Since these will be fit to data from 1976-1999, this may provide a better approximation to Krueger (2017) by extrapolating only the recent trend at each age/gender group. 

3. $\alpha(a,g,c)+\beta(a,g,c)c$ - This is conceptually similar to the above, but forecasts trends by cohort rather than time. This is likely to be close to #2, since cohort is a linear function of age and year, but not identical since the SE kernel is nonlinear, which may induce a slightly different structure. 

4. $\alpha_1(a,g,t)+\alpha_2(c)+\beta(a,g,t)t$ - This adds a stationary cohort effect to #2, which may capture additional variation in cohort-specific participation patterns. 

5. $\alpha(a,g)+\beta(c)c$ - This approximates the age/gender/cohort fixed effects model. Age and gender contribute additively with a cohort term, the latter of which takes the form of a time-varying linear trend in order to extrapolate to new cohorts.

6. $\alpha(a,g,t)+\beta_1(a,g,t)t+\beta_2(a,g)UGap_t$ - This adds a cyclical term to #2, linear in the unemployment gap, $UGap_t$, allowing the coefficient on this term to vary smoothly across age and gender. The unemployment gap is computed as the overall unemployment rate minus a long-term trend estimated non-parametrically with a biweight kernel with bandwidth 120. (The unemployment gap is the same for all age/gender cells, but the coefficient on it is allowed to vary.)

7. $\alpha(a,g,t)+\beta_1(a,g,t)t+\beta_2(a,g)UGap_t+\beta_3(a,g)UGap_{t-1}+\beta_4(a,g)UGap_{t-2}$ - This is the same as #6, but adding in the one and two year lags of the unemployment gap as well.

## Estimation

To estimate and forecast using each kernel, I compile a dataset of LFPR by age, gender, and year from the CPS. Since the LFPR is constrained between 0 and 1, I transform this into an unconstrained outcome by passing each observation through the logistic function. I also normalize the outcome and all covariates into z-scores to facilitate easier estimation. 

I begin by estimating the hyperparameters for each kernel. Each kernel is governed by several hyperparameters controlling the variances and length scales of each component, as well as the overall noise in the data, $\sigma$. To estimate the hyperparameters, I choose the value which maximizes the marginal likelihood over the 1976-1999 period. 

Using the estimate of the hyperparameters, I compute the predictions over the 2000-2016 period, and then un-normalize and exponentiate them. These are plotted along with the full data below. 

In [3]:
execfile("scripts/draw_testkernel_plots_plotly1.py")

IntRangeSlider(value=(16, 80), continuous_update=False, description=u'Age Range:', max=80, min=16)

IntRangeSlider(value=(1976, 2016), continuous_update=False, description=u'Year Range:', max=2016, min=1976)

Box(children=(Label(value=u'Include Forecast with Kernel:'), Dropdown(index=7, options=OrderedDict([('1 - Krueger', 1), ('2 - Local Trends', 2), ('3 - Cohort Trends', 3), ('4 - Local Trends + Cohort', 4), ('5 - Age/Gender/Cohort FE', 5), ('6 - Local Trends + UR', 6), ('7 - Local Trends + UR & Lags', 7), ('None', 0)]), value=0)))

In [10]:
execfile("scripts/draw_testkernel_plots_plotly2.py")